Considere:

$$
\begin{align*}
    \begin{bmatrix}
        1 & 2 & 3\\
        3 & 2 & 1\\
        1 & 1 & -1
    \end{bmatrix}
    \mathbf{x} &=
    \begin{bmatrix}
        1\\
        1\\
        1
    \end{bmatrix}.
\end{align*}
$$

1. Construya el sub-espacio de Krylov *no-ortonormalizado* asociado.
2. Construya el sub-espacio de Krylov *ortonormalizado* asociado.
3. Construya la descomposición de Hessemberg asociada.
4. Determine el sistema de ecuaciones *equivalente* a resolver con GMRes.

In [1]:
import numpy as np

In [11]:
A = np.array([[1.,2.,3.],[3.,2.,1.],[1.,1.,-1.]])
b = np.array([1.,1.,1.])

$b_1 = b$, $b_2 = A\,b$, $b_3 = A^2\,b = A\,(A\,b) = A\,b_2$

In [12]:
b1 = b
b2 = np.dot(A,b1)
b3 = np.dot(A,b2)
print(b1,b2,b3)

[1. 1. 1.] [6. 6. 1.] [21. 31. 11.]


$q_1 = b/\lVert b \rVert$

Procedimiento para $q_2$

$A\,q_1 = h_{11}\,q_1 + h_{21}\,q_2$ (multiplicar por la izquierda $q_1^*$)

$q_1^*(A\,q_1) = h_{11}$

$y_2 = A\,q_1 - h_{11}\,q_1 = h_{21}\,q_2$

$h_{21} = \lVert y_2 \rVert$, $q_2 = y_2 / h_{21}$

Procedimiento para $q_3$

$A\,q_2 = h_{12}\,q_1 + h_{22}\,q_2 + h_{32}\,q_3$ (multiplicar por la izquierda $q_1^*$)

$q_1^*(A\,q_2) = h_{12}$

$A\,q_2 = h_{12}\,q_1 + h_{22}\,q_2 + h_{32}\,q_3$ (multiplicar por la izquierda $q_2*$)

$q_2^*(A\,q_2) = h_{22}$

$y_3 = A\,q_2 - h_{12}\,q_1 - h_{22}\,q_2 = h_{32}\,q_3$

$h_{32} = \lVert y_3 \rVert$, $q_3 = y_3 / h_{32}$









In [26]:
q1 = b/np.linalg.norm(b)
y1 = np.dot(A,q1)
h11 = np.dot(q1,y1)

y2 = y1 - h11*q1
h21 = np.linalg.norm(y2)
q2 = y2 / h21

p2 = np.dot(A,q2)
h12 = np.dot(q1,p2)
h22 = np.dot(q2,p2)
y3 = p2 - h12*q1 - h22*q2
h32 = np.linalg.norm(y3)
q3 = y3 / h32

1.0

In [42]:
Q = np.zeros((3,3))
Q[:,0] = q1
Q[:,1] = q2
Q[:,2] = q3

H = np.array([[h11,h12],[h21,h22],[0.,h32]])

L = A@Q[:,:2]
R = Q@H

$H \, \begin{bmatrix}c_1 \\ c_2\end{bmatrix} = \begin{bmatrix}\lVert b \rVert \\0 \\ 0\end{bmatrix}$

$Qh,Rh = QR(H)$

$Rh\,c = Qh^*\begin{bmatrix}\lVert b \rVert \\0 \\ 0\end{bmatrix}$

In [52]:
bh = np.zeros((3,1))
bh[0] = np.linalg.norm(b)

Qh,Rh = np.linalg.qr(H,mode="reduced")
bqr = np.dot(Qh.T,bh)
c = np.linalg.solve(Rh,bqr)

array([[0.29921072],
       [0.23839316]])

In [53]:
x = np.linalg.solve(A,b)
x

array([-0.25,  1.  , -0.25])

In [56]:
xc = c[0]*q1 + c[1]*q2
np.linalg.norm(b - np.dot(A,xc))

0.6041220933301769